In [1]:
!python -m spacy download en_core_web_lg
import spacy
import tensorflow as tf
import re
import spacy
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import  Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from google.colab import drive

drive.mount("/content/drive")

import numpy as np
import os
import time
nlp = spacy.load("en_core_web_lg")
file = open("/content/drive/MyDrive/466 data/en_tech_train.txt", "rb")
train_set = file.read().decode(encoding='utf-8')
file = open("/content/drive/MyDrive/466 data/en_tech_test.txt", "rb")
test_set = file.read().decode(encoding='utf-8')
file = open("/content/drive/MyDrive/466 data/en_tech_validate.txt", "rb")
validate_set = file.read().decode(encoding='utf-8')

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
def predict_single(model, tokenizer, seed_text):
  
  predicted_word = None
  encoded = tokenizer.texts_to_sequences([seed_text])[0]
  encoded = pad_sequences([encoded], maxlen = 4, truncating='pre')

  temp = model.predict(encoded)

  y_predict = np.argmax(temp, axis=-1)

  for word, index in tokenizer.word_index.items():
      if index == y_predict:
        predicted_word = word
        break
  return str(predicted_word)

In [3]:
def generate_text(model, tokenizer, text_seq_len, seed_text, n_words):
  text = []
  word_distribution = []

  for _ in range(n_words):
    encoded = tokenizer.texts_to_sequences([seed_text])[0]
    #encoded = pad_sequences([encoded], maxlen = 4, truncating='pre')

    temp = model.predict(encoded)

    y_predict = np.argmax(temp, axis=-1)
    #print(y_predict)

    word_list = []
    max_index = y_predict
    for x in range(4):
      temp[0][max_index] = -1
      max_index = np.argmax(temp, axis=-1)
      word_list.append(max_index)

    predicted_word = ''
    temp = model.predict(encoded)

    for word, index in tokenizer.word_index.items():
      if index == y_predict:
        predicted_word = word
        break
    seed_text = seed_text + ' ' + predicted_word
    text.append(predicted_word)

    distribution = [(predicted_word, temp[0][y_predict][0])]
    for predicted in word_list:
      predicted_word = ''
      for word, index in tokenizer.word_index.items():
        if index == predicted:
          predicted_word = word
          break
      distribution.append((predicted_word, temp[0][predicted][0]))
    word_distribution.append(distribution)

  return ' '.join(text), word_distribution


In [4]:
def validate(validate_sentences, nlp):
  new = []
  for i in range(len(validate_sentences)):
    new.append(validate_sentences[i].split())
  validate_sequences = np.array(new)
  validate_X, validate_y = validate_sequences[:, :-1], validate_sequences[:, -1]
  average_accuracy = 0
  num = 0
  for i in range(len(validate_X)):
    prediction = predict_single(model, tokenizer, ' '.join(validate_X[i]))
    x = nlp(prediction)
    y = nlp(str(validate_y[i]))

    if (x[0].has_vector) and (y[0].has_vector):
      average_accuracy += nlp(prediction).similarity(nlp(str(validate_y[i])))
      num += 1

  average_accuracy /= num
  return average_accuracy

In [ ]:
processed_train_set = train_set.split("\n")
processed_test_set = test_set.split("\n")
processed_validate_set = validate_set.split("\n")

input_size = 5
train_sentences = []
test_sentences = []
validate_sentences = []
for line in processed_train_set:
    temp = line.split()
    for i in range(input_size, len(temp)):
        seq = temp[i - input_size:i]
        temp2 = " ".join(seq)
        train_sentences.append(temp2)
train_sentences[0:10]
for line in processed_test_set:
    temp = line.split()
    for i in range(input_size, len(temp)):
        seq = temp[i - input_size:i]
        temp2 = " ".join(seq)
        test_sentences.append(temp2)
test_sentences[0:10]
for line in processed_validate_set:
    temp = line.split()
    for i in range(input_size, len(temp)):
        seq = temp[i - input_size:i]
        temp2 = " ".join(seq)
        validate_sentences.append(temp2)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_sentences)
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_sequences = np.array(train_sequences)
train_X, train_y = train_sequences[:, :-1], train_sequences[:, -1]

vocab_size = len(tokenizer.word_index) + 1


train_y = to_categorical(train_y, num_classes = vocab_size)
# sequences = np.array(sequences)
# print(sequences.shape)
model = Sequential()
model.add(Embedding(vocab_size, 256))
# model.add(GRU(512, return_sequences=True))
model.add(GRU(512))
model.add(Dense(1024, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
model.summary()
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_X,train_y, batch_size=256, epochs=20)
accuracy = validate(validate_sentences, nlp)
print(accuracy)


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 256)         4601600   
_________________________________________________________________
gru_3 (GRU)                  (None, 512)               1182720   
_________________________________________________________________
dense_6 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dense_7 (Dense)              (None, 17975)             18424375  
Total params: 24,734,007
Trainable params: 24,734,007
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
801/801 [==============================] - 36s 42ms/step - loss: 7.3767 - accuracy: 0.0700
Epoch 2/20
801/801 [==============================] - 34s 42ms/step - loss: 6.1322 - accuracy: 0.1378
Epoch 3/20
801/801 [================